In [ ]:
from datetime import datetime
import numpy as np
import pandas as pd
from collections import defaultdict
import matplotlib.pyplot as plt


In [ ]:
dai = pd.read_csv('podatki/delovno-aktivno-obcina-izobrazba-spol.csv',sep=' ',encoding='cp1250')
#občine, leto, dosežena izobrazba, spol 
#link: https://pxweb.stat.si/SiStatData/pxweb/sl/Data/-/0764721S.PX
dai

In [ ]:
dsa = pd.read_csv('podatki/delovno-aktivno-status-aktivnosti7.csv',sep=' ',encoding='cp1250')
#Delovno aktivno prebivalstvo po statusu aktivnosti in občinah delovnega mesta, Slovenija, mesečno
#Link: https://pxweb.stat.si/SiStatData/pxweb/sl/Data/-/0700941S.px
dsa

In [ ]:
ppo = pd.read_csv('podatki/prebivalstvo-po-občinah.csv',sep=' ',encoding='cp1250', low_memory=False)
#Prebivalstvo po velikih in petletnih starostnih skupinah in spolu, občine, Slovenija, polletno#Link: https://pxweb.stat.si/SiStatData/pxweb/sl/Data/-/0952752S.px
#Link: https://pxweb.stat.si/SiStatData/pxweb/sl/Data/-/05C4004S.px
ppo

In [ ]:
# Slovar - ključ = leto
slovar2 = {} 
obc = [] #občine
delez_kmetje = []
leto = 2005 # začetno leto

komplet = 0 # Ko doseze tri pregledamo vse tri vrstice za obcino
for i in range(2, dsa.shape[1] - 1): # 10 lahko spremenimo da dobimo drugo vrednost (npr osnovnešole itd)
    for index, row in dsa.iterrows():        
        #if (row[i] == "z"):
        #    print(leto, row[0], row[i])
        if (row[i] == "-" or row[i] == "z"  or row[0] == "SLOVENIJA"): ## z ker kostel in Osilnica leta 2019 vrednost 'z'
            continue # podatek manjka oziroma je za celotno Slovenijo zato preskočimo..
        komplet += 1
        
        if (komplet == 1):
            # prva vrstica - skupno delovno aktivno
            skupno = row[i]
        
        if (komplet == 2):
            delez = int(row[i]) / int(skupno)
            #print(row[0], delez)
            komplet = 0
            obc.append(row[0])
            delez_kmetje.append(delez)
            
    slovar2[leto] = [obc, delez_kmetje]
   
    print(leto, len(slovar2[leto][0])) # Vidimo koliko občin ima podatke za določeno leto
    
    leto+=1
    
    # Resetiram..
    obc = []
    delez_kmetje = []


In [ ]:
# Uredim po abecedi da bom lahko združil tabeli
sdai = dai.sort_values(by=["OBČINE"], ascending=True)
sdai

In [ ]:
# Dodamo skupne podatke o prebivalstvu po občinah(SKUPAJ)
ppo_S = ppo.head(213)
ppo_S = ppo_S.drop(['SPOL'], axis = 1) #Znembim se uvodnega stolpca
mergedS = sdai.merge(ppo_S) 

# Dodamo podatke o prebivalstvu po občinah za moške
ppo_M = ppo.iloc[213:426]
ppo_M = ppo_M.drop(['SPOL'], axis = 1) #Znembim se uvodnega stolpca
#ppo_M.index = range(213)
mergedM = sdai.merge(ppo_M)

# Dodamo podatke o prebivalstvu po občinah za ženske
ppo_Z = ppo.tail(213)
ppo_Z = ppo_Z.drop(['SPOL'], axis = 1) #Znembim se uvodnega stolpca
mergedZ = sdai.merge(ppo_Z)




In [ ]:
# Testiraš vrednosti kraja - vidiš razporeditev

#for index, row in mergedS.iterrows():
#     if row[0] == "Ljubljana":
#        for i in range(1381):
#            print(i, row[i])

#        break


In [ ]:
sdai.iloc[:,0:16] # Vidimo podatke za eno leto, 15 stolpcev na leto.. 255 / 15 = 17 -> 17 let [2005 - 2021]


In [ ]:
sdai.iloc[:,10::15] # Vidimo podatke za vse leta za Visokošolsko SKUPAJ (M + Ž)
# Če 10 spremeniš v 7 dobiš za srednješolsko, če v 4 pa za osnovnošolsko

In [ ]:
## FOR LOOP ZA DOBIVANJE VSEH VREDNOSTI (VSA LETA) DOLOČENEGA STOLPCA 
for index, row in sdai.iterrows():
    print(row[0]) # Izpišemo za kateri kraj so ti podatki.
    
    for i in range(10, len(row), 15): # 10 lahko spremenimo da dobimo drugo vrednost
        print(i, row[i])
    break; # Breakamo po prvi vrstici(demonstracija delovanja)

In [ ]:
## Potem nisem uporabil..

# Poglejmo kje se nahajajo podatki o starosti, ki jih bomo uporabili za normalizacijo
#print(mergedS.columns.get_loc("2005H2 15-64 let"))
#print(mergedS.columns.get_loc("2006H2 15-64 let"))
#print(mergedS.columns.get_loc("2007H2 15-64 let"))
#print(mergedS.columns.get_loc("2008H2 15-64 let"))
#print(mergedS.columns.get_loc("2009H2 15-64 let"))
#print(mergedS.columns.get_loc("2010H2 15-64 let"))
#print(mergedS.columns.get_loc("2011H2 15-64 let"))
#print(mergedS.columns.get_loc("2012H2 15-64 let"))
#print(mergedS.columns.get_loc("2013H2 15-64 let"))
#print(mergedS.columns.get_loc("2014H2 15-64 let"))
#print(mergedS.columns.get_loc("2015H2 15-64 let"))
#print(mergedS.columns.get_loc("2016H2 15-64 let"))
#print(mergedS.columns.get_loc("2017H2 15-64 let"))
#print(mergedS.columns.get_loc("2018H2 15-64 let"))
#print(mergedS.columns.get_loc("2019H2 15-64 let"))
#print(mergedS.columns.get_loc("2020H2 15-64 let"))
#print(mergedS.columns.get_loc("2021H2 15-64 let"))
# Vidimo da začnemo v 558 stolpcu in se premikamo s korakom 50 za pridobitve podatka naslednjega leta

In [ ]:
# Slovar - ključ = leto
slovar = {} 
LJ_vs = []
LJ_ss = []
LJ_os = []
MB_vs = []
MB_ss = []
MB_os = []
KP_vs = []
KP_ss = []
KP_os = []
CE_vs = []
CE_ss = []
CE_os = []
BL_vs = []
BL_ss = []
BL_os = []

obc = [] #občine
## VISOKOŠOLSKA
VS_v = [] #vrednosti
VS_n = [] #normalizirane vrednosti
# SREDNJEŠOLSKA
SS_v = [] #vrednosti
SS_n = [] #normalizirane vrednosti
# OSNOVNOŠOLSKA
OS_v = [] #vrednosti
OS_n = [] #normalizirane vrednosti
leto = 2005 # začetno leto
j = 558 # index ki kaže na stolpec s podatki o prebivalstvu med 15-64 let (558 -> za leto 2005)
for i in range(10, 256, 15): # 10 lahko spremenimo da dobimo drugo vrednost (npr osnovnešole itd)
    for index, row in mergedS.iterrows():        
        
        if (row[i] == "-" or row[i] == "z" or row[0] == "SLOVENIJA"): ## z ker kostel in Osilnica leta 2019 vrednost 'z'
            continue # podatek manjka oziroma je za celotno Slovenijo zato preskočimo..
        
        vsota = int(row[i]) + int(row[i-3]) + int(row[i-6])
        if (row[0] == "Ljubljana"):
            
            LJ_vs.append(int(row[i]) / vsota)
            LJ_ss.append(int(row[i-3]) / vsota)
            LJ_os.append(int(row[i-6]) / vsota)
        elif (row[0] == "Maribor"):
            MB_vs.append(int(row[i]) / vsota)
            MB_ss.append(int(row[i-3]) / vsota)
            MB_os.append(int(row[i-6]) / vsota)
        elif (row[0] == "Koper/Capodistria"):
            KP_vs.append(int(row[i]) / vsota)
            KP_ss.append(int(row[i-3]) / vsota)
            KP_os.append(int(row[i-6]) / vsota)
        elif (row[0] == "Celje"):
            CE_vs.append(int(row[i]) / vsota)
            CE_ss.append(int(row[i-3]) / vsota)
            CE_os.append(int(row[i-6]) / vsota)  
        elif (row[0] == "Bled"):
            BL_vs.append(int(row[i]) / vsota)
            BL_ss.append(int(row[i-3]) / vsota)
            BL_os.append(int(row[i-6]) / vsota)
        
        obc.append(row[0])
        VS_v.append(int(row[i]))
        VS_n.append(int(row[i]) / vsota)
        SS_v.append(int(row[i-3]))
        SS_n.append(int(row[i-3]) / vsota)
        OS_v.append(int(row[i-6]))
        OS_n.append(int(row[i-6]) / vsota)
    
    slovar[leto] = [obc, VS_v, VS_n, SS_v, SS_n, OS_v, OS_n]
    j += 50
   
    print(leto, len(slovar[leto][0])) # Vidimo koliko občin ima podatke za določeno leto
    
    leto+=1
    
    # Resetiram..
    VS_v = []
    obc = []
    VS_n = []
    SS_v = []
    SS_n = []
    OS_v = []
    OS_n = []
      


In [ ]:
# Za povečanje velikosti izrisa grafov

from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 100em; }</style>"))

In [ ]:
from scipy.stats.stats import pearsonr   
for i in range(2005, 2016):
    font = {'family' : 'arial',
        'weight' : 'normal',
        'size'   : 10}

    plt.rc('font', **font)
    print(i)
    plt.figure(figsize=(50,140))
    mesta2 = slovar2[i][0]
    delez_kmetov = slovar2[i][1]
    
    mesta = slovar[i][0]
    statis_VS = slovar[i][2]
    statis_SS = slovar[i][4]
    statis_OS = slovar[i][6]
   
    print("VS:")
    print(pearsonr(delez_kmetov,statis_VS))
    print("SS:")
    print(pearsonr(delez_kmetov,statis_SS))
    print("OS:")
    print(pearsonr(delez_kmetov,statis_OS))
    df_list = pd.DataFrame(
    {'statis_VS': statis_VS,
     'statis_SS': statis_SS,
     'statis_OS': statis_OS,
     'delez_kmetov': delez_kmetov
    })
    
    
    df_list.plot.scatter(x = "statis_VS", y = "delez_kmetov", c="blue", figsize=(10,5)) 
    plt.title(f"Scatter - delež kmetov, delež prebivalstva z visokošolsko izobrazbo. Leto - {i}")
    plt.xlabel("Delež delovno aktivnega prebivalstva z visokošolsko izobrazbo")
    plt.ylabel("Delež delovno aktivnega prebivalstva samozaposlenih kot kmetje")
    z = np.polyfit(statis_VS, delez_kmetov, 1)
    p = np.poly1d(z)
    plt.plot(statis_VS,p(statis_VS),"r--")
    
    df_list.plot.scatter(x = "statis_SS", y = "delez_kmetov", c="green", figsize=(10,5)) 
    plt.title(f"Scatter - delež kmetov, delež prebivalstva s srednješolsko izobrazbo. Leto - {i}")
    plt.xlabel("Delež delovno aktivnega prebivalstva s srednješolsko izobrazbo")
    plt.ylabel("Delež delovno aktivnega prebivalstva samozaposlenih kot kmetje")
    z = np.polyfit(statis_SS, delez_kmetov, 1)
    p = np.poly1d(z)
    plt.plot(statis_SS,p(statis_SS),"r--")
    
    df_list.plot.scatter(x = "statis_OS", y = "delez_kmetov", c="orange",figsize=(10,5)) 
    plt.title(f"Scatter - delež kmetov, delež prebivalstva z osnovnošolsko izobrazbo. Leto - {i}")
    plt.xlabel("Delež delovno aktivnega prebivalstva z osnovnošolsko izobrazbo")
    plt.ylabel("Delež delovno aktivnega prebivalstva samozaposlenih kot kmetje")
    z = np.polyfit(statis_OS, delez_kmetov, 1)
    p = np.poly1d(z)
    plt.plot(statis_OS,p(statis_OS),"r--")
    
    
    plt.show()

    


In [ ]:
## EXCEL OUTPUT (Za zemljevide)
#for i in range (2005, 2022):
#    os = zip(slovar[i][0],slovar[i][6])
#    pd.DataFrame(os).to_excel('os' + str(i) + '.xlsx', header=False, index=False)
#    ss = zip(slovar[i][0],slovar[i][4])
#    pd.DataFrame(ss).to_excel('ss' + str(i) + '.xlsx', header=False, index=False)
#    vs = zip(slovar[i][0],slovar[i][2])
#    pd.DataFrame(vs).to_excel('vs' + str(i) + '.xlsx', header=False, index=False)

In [ ]:
### VISOKOŠOLSKA
### ABSOLUTNE VREDNOSTI ###

for i in range(2005, 2022):
    font = {'family' : 'arial',
        'weight' : 'normal',
        'size'   : 40}

    plt.rc('font', **font)

    plt.figure(figsize=(50,140))

    plt.title(f"Absolutno število - Visokošolska izobrazba Leto - {i}")
    mesta = slovar[i][0]
    statis_VS = slovar[i][1] 
    mesta_sorted = [x for _,x in sorted(zip(statis_VS, mesta))]
    statis_VS_sorted = sorted(statis_VS)

    cc=['colors']*len(statis_VS_sorted)
    for n,val in enumerate(statis_VS_sorted):
        if val<500:
            cc[n]='lightskyblue'
        elif val>=500 and val<1000:
            cc[n]='deepskyblue'
        elif val>=1000 and val<2500:
            cc[n]='dodgerblue'
        elif val >= 2500 and val < 4500:
            cc[n] ='royalblue'
        else:
            cc[n] ='navy'

    plt.barh(range(len(mesta_sorted)), statis_VS_sorted, color=cc, 
         tick_label=mesta_sorted)

    plt.show()


In [ ]:

### VISOKOŠOLSKA
### NORMALIZIRANE VREDNOSTI GLEDE NA POPULACIJO ###


for i in range(2005, 2022):

    font = {'family' : 'arial',
            'weight' : 'normal',
            'size'   : 40}

    plt.rc('font', **font)

    plt.figure(figsize=(50,140))
    
    plt.title(f"Normalizirano glede na populacijo - Visokošolska izobrazba Leto - {i}")
    mesta = slovar[i][0]
    normal_VS = slovar[i][2] 

    mesta_sorted = [x for _,x in sorted(zip(normal_VS, mesta))]
    normal_VS_sorted = sorted(normal_VS)

    cc=['colors']*len(normal_VS_sorted)
    for n,val in enumerate(normal_VS_sorted):
        if val < 0.05:
            cc[n]='aquamarine'
        elif val >= 0.05 and val<0.1:
            cc[n]='turquoise'
        elif val >= 0.1 and val<0.15:
            cc[n]='darkcyan'
        elif val >= 0.15 and val<0.20:
            cc[n]='lavender'
        elif val >= 0.20 and val < 0.25:
            cc[n] ='lightsteelblue'
        elif val >= 0.25 and val < 0.30:
            cc[n] ='lightskyblue'
        elif val >= 0.30 and val < 0.35:
            cc[n] ='deepskyblue'
        elif val >= 0.35 and val < 0.40:
            cc[n] ='dodgerblue'
        elif val >= 0.40 and val < 0.45:
            cc[n] ='royalblue' 
        else:
            cc[n] ='navy'

    plt.barh(range(len(mesta_sorted)), normal_VS_sorted, color=cc, 
         tick_label=mesta_sorted)

    plt.show()


In [ ]:
### SREDNJEŠOLSKA
### ABSOLUTNE VREDNOSTI ###

for i in range(2005, 2022):
    font = {'family' : 'arial',
        'weight' : 'normal',
        'size'   : 40}

    plt.rc('font', **font)

    plt.figure(figsize=(50,140))

    plt.title(f"Absolutno število - Srednješolska izobrazba Leto - {i}")
    mesta = slovar[i][0]
    statis_VS = slovar[i][3] 
    mesta_sorted = [x for _,x in sorted(zip(statis_VS, mesta))]
    statis_VS_sorted = sorted(statis_VS)

    cc=['colors']*len(statis_VS_sorted)
    for n,val in enumerate(statis_VS_sorted):
        if val<500:
            cc[n]='lightskyblue'
        elif val>=500 and val<1000:
            cc[n]='deepskyblue'
        elif val>=1000 and val<2500:
            cc[n]='dodgerblue'
        elif val >= 2500 and val < 4500:
            cc[n] ='royalblue'
        else:
            cc[n] ='navy'


    plt.barh(range(len(mesta_sorted)), statis_VS_sorted, color=cc, 
         tick_label=mesta_sorted)

    plt.show()


In [ ]:

### SREDNJEŠOLSKA
### NORMALIZIRANE VREDNOSTI GLEDE NA POPULACIJO ###


for i in range(2005, 2022):

    font = {'family' : 'arial',
            'weight' : 'normal',
            'size'   : 40}

    plt.rc('font', **font)

    plt.figure(figsize=(50,140))
    
    plt.title(f"Normalizirano glede na populacijo - Srednješolska izobrazba Leto - {i}")
    mesta = slovar[i][0]
    normal_VS = slovar[i][4] 

    mesta_sorted = [x for _,x in sorted(zip(normal_VS, mesta))]
    normal_VS_sorted = sorted(normal_VS)

    cc=['colors']*len(normal_VS_sorted)
    for n,val in enumerate(normal_VS_sorted):
        if val < 0.05:
            cc[n]='aquamarine'
        elif val >= 0.05 and val<0.1:
            cc[n]='turquoise'
        elif val >= 0.1 and val<0.15:
            cc[n]='darkcyan'
        elif val >= 0.15 and val<0.20:
            cc[n]='lavender'
        elif val >= 0.20 and val < 0.25:
            cc[n] ='lightsteelblue'
        elif val >= 0.25 and val < 0.30:
            cc[n] ='lightskyblue'
        elif val >= 0.30 and val < 0.35:
            cc[n] ='deepskyblue'
        elif val >= 0.35 and val < 0.40:
            cc[n] ='dodgerblue'
        elif val >= 0.40 and val < 0.45:
            cc[n] ='royalblue' 
        else:
            cc[n] ='navy'

    plt.barh(range(len(mesta_sorted)), normal_VS_sorted, color=cc, 
         tick_label=mesta_sorted)

    plt.show()


In [ ]:
### OSNOVNOŠOLSKA
### ABSOLUTNE VREDNOSTI ###

for i in range(2005, 2022):
    font = {'family' : 'arial',
        'weight' : 'normal',
        'size'   : 40}

    plt.rc('font', **font)

    plt.figure(figsize=(50,140))

    plt.title(f"Absolutno število - Osnovnošolska izobrazba Leto - {i}")
    mesta = slovar[i][0]
    statis_VS = slovar[i][5] 
    mesta_sorted = [x for _,x in sorted(zip(statis_VS, mesta))]
    statis_VS_sorted = sorted(statis_VS)

    cc=['colors']*len(statis_VS_sorted)
    for n,val in enumerate(statis_VS_sorted):
        if val<500:
            cc[n]='lightskyblue'
        elif val>=500 and val<1000:
            cc[n]='deepskyblue'
        elif val>=1000 and val<2500:
            cc[n]='dodgerblue'
        elif val >= 2500 and val < 4500:
            cc[n] ='royalblue'
        else:
            cc[n] ='navy'


    plt.barh(range(len(mesta_sorted)), statis_VS_sorted, color=cc, 
         tick_label=mesta_sorted)
    plt.show()


In [ ]:

### OSNOVNOŠOLSKA
### NORMALIZIRANE VREDNOSTI GLEDE NA POPULACIJO ###


for i in range(2005, 2022):

    font = {'family' : 'arial',
            'weight' : 'normal',
            'size'   : 40}

    plt.rc('font', **font)

    plt.figure(figsize=(50,140))
    
    plt.title(f"Normalizirano glede na populacijo - Osnovnošolska izobrazba Leto - {i}")
    mesta = slovar[i][0]
    normal_VS = slovar[i][6] 

    mesta_sorted = [x for _,x in sorted(zip(normal_VS, mesta))]
    normal_VS_sorted = sorted(normal_VS)

    cc=['colors']*len(normal_VS_sorted)
    for n,val in enumerate(normal_VS_sorted):
        if val < 0.05:
            cc[n]='aquamarine'
        elif val >= 0.05 and val<0.1:
            cc[n]='turquoise'
        elif val >= 0.1 and val<0.15:
            cc[n]='darkcyan'
        elif val >= 0.15 and val<0.20:
            cc[n]='lavender'
        elif val >= 0.20 and val < 0.25:
            cc[n] ='lightsteelblue'
        elif val >= 0.25 and val < 0.30:
            cc[n] ='lightskyblue'
        elif val >= 0.30 and val < 0.35:
            cc[n] ='deepskyblue'
        elif val >= 0.35 and val < 0.40:
            cc[n] ='dodgerblue'
        elif val >= 0.40 and val < 0.45:
            cc[n] ='royalblue' 
        else:
            cc[n] ='navy'

    plt.barh(range(len(mesta_sorted)), normal_VS_sorted, color=cc, 
         tick_label=mesta_sorted)

    plt.show()


In [ ]:
def rolling_avg_list(velikost_okna, seznam):
    vsota = 0
    i = 0
    seznam1 = []
    sezrat = []
    for element in seznam:
        i+=1
        sezrat.append(element)
        vsota+=element
        if (i>=velikost_okna):
            seznam1.append(vsota/velikost_okna)
            vsota-= sezrat.pop(0)
            
    return seznam1


In [ ]:
LJ_vs_pet = rolling_avg_list(5, LJ_vs)
LJ_ss_pet = rolling_avg_list(5, LJ_ss)
LJ_os_pet = rolling_avg_list(5, LJ_os)

MB_vs_pet = rolling_avg_list(5, MB_vs)
MB_ss_pet = rolling_avg_list(5, MB_ss)
MB_os_pet = rolling_avg_list(5, MB_os)

KP_vs_pet = rolling_avg_list(5, KP_vs)
KP_ss_pet = rolling_avg_list(5, KP_ss)
KP_os_pet = rolling_avg_list(5, KP_os)

CE_vs_pet = rolling_avg_list(5, CE_vs)
CE_ss_pet = rolling_avg_list(5, CE_ss)
CE_os_pet = rolling_avg_list(5, CE_os)

BL_vs_pet = rolling_avg_list(5, BL_vs)
BL_ss_pet = rolling_avg_list(5, BL_ss)
BL_os_pet = rolling_avg_list(5, BL_os)


In [ ]:
### ROLLING AVERAGE - LJUBLJANA, MARIBOR, KOPER, BLED, ILIRSKA BISTRICA ###



font = {'family' : 'arial',
            'weight' : 'normal',
            'size'   : 16}

plt.rc('font', **font)

# LJUBLJANA 
plt.figure(figsize=(15, 10))
plt.xlabel("Leto")
plt.ylabel("Delež prebivalstva občine")
plt.title("Ljubljana - Graf deleža delovno aktivnega prebivalstva občine z določeno stopnjo izobrazbe z oknom velikosti 5")

plt.plot(list(range(2009, 2022)), LJ_vs_pet, color='blue', label="Ljubljana Visokošolska izobrazba")
plt.plot(list(range(2009, 2022)), LJ_ss_pet, color='green', label="Ljubljana Srednješolska izobrazba")
plt.plot(list(range(2009, 2022)), LJ_os_pet, color='fuchsia', label="Ljubljana Osnovnošolska izobrazba")
plt.legend()
plt.show()

# MARIBOR
plt.figure(figsize=(15, 10))
plt.xlabel("Leto")
plt.ylabel("Delež prebivalstva občine")
plt.title("Maribor - Graf deleža delovno aktivnega prebivalstva občine z določeno stopnjo izobrazbe z oknom velikosti 5")

plt.plot(list(range(2009, 2022)), MB_vs_pet, color='blue', label="Maribor Visokošolska izobrazba")
plt.plot(list(range(2009, 2022)), MB_ss_pet, color='green', label="Maribor Srednješolska izobrazba")
plt.plot(list(range(2009, 2022)), MB_os_pet, color='fuchsia', label="Maribor Osnovnošolska izobrazba")
plt.legend()
plt.show()

# KOPER
plt.figure(figsize=(15, 10))
plt.xlabel("Leto")
plt.ylabel("Delež prebivalstva občine")
plt.title("Koper - Graf deleža delovno aktivnega prebivalstva občine z določeno stopnjo izobrazbe z oknom velikosti 5")

plt.plot(list(range(2009, 2022)), KP_vs_pet, color='blue', label="Koper Visokošolska izobrazba")
plt.plot(list(range(2009, 2022)), KP_ss_pet, color='green', label="Koper Srednješolska izobrazba")
plt.plot(list(range(2009, 2022)), KP_os_pet, color='fuchsia', label="Koper Osnovnošolska izobrazba")
plt.legend()
plt.show()

# CELJE
plt.figure(figsize=(15, 10))
plt.xlabel("Leto")
plt.ylabel("Delež prebivalstva občine")
plt.title("Celje - Graf deleža delovno aktivnega prebivalstva občine z določeno stopnjo izobrazbe z oknom velikosti 5")

plt.plot(list(range(2009, 2022)), CE_vs_pet, color='blue', label="Celje Visokošolska izobrazba")
plt.plot(list(range(2009, 2022)), CE_ss_pet, color='green', label="Celje Srednješolska izobrazba")
plt.plot(list(range(2009, 2022)), CE_os_pet, color='fuchsia', label="Celje Osnovnošolska izobrazba")
plt.legend()
plt.show()

# BLED
plt.figure(figsize=(15, 10))
plt.xlabel("Leto")
plt.ylabel("Delež prebivalstva občine")
plt.title("Bled - Graf deleža delovno aktivnega prebivalstva občine z določeno stopnjo izobrazbe z oknom velikosti 5")

plt.plot(list(range(2009, 2022)), BL_vs_pet, color='blue', label="Bled Visokošolska izobrazba")
plt.plot(list(range(2009, 2022)), BL_ss_pet, color='green', label="Bled Srednješolska izobrazba")
plt.plot(list(range(2009, 2022)), BL_os_pet, color='fuchsia', label="Bled Osnovnošolska izobrazba")
plt.legend()
plt.show()


# PRIMERJAVA VISOKOŠOLSKE

plt.figure(figsize=(15, 10))
plt.xlabel("Leto")
plt.ylabel("Delež prebivalstva občine")
plt.title("Graf deleža delovno aktivnega prebivalstva občine z visokošolsko stopnjo izobrazbe z oknom velikosti 5")

plt.plot(list(range(2009, 2022)), LJ_vs_pet, color='blue', label="Ljubljana visokošolska izobrazba")
plt.plot(list(range(2009, 2022)), MB_vs_pet, color='green', label="Maribor visokošolska izobrazba")
plt.plot(list(range(2009, 2022)), KP_vs_pet, color='red', label="Koper visokošolska izobrazba")
plt.plot(list(range(2009, 2022)), CE_vs_pet, color='orange', label="Celje visokošolska izobrazba")
plt.plot(list(range(2009, 2022)), BL_vs_pet, color='violet', label="Bled visokošolska izobrazba")
plt.legend()
plt.show()


# PRIMERJAVA SREDNJEŠOLSKE

plt.figure(figsize=(15, 10))
plt.xlabel("Leto")
plt.ylabel("Delež prebivalstva občine")
plt.title("Graf deleža delovno aktivnega prebivalstva občine z srednješolsko stopnjo izobrazbe z oknom velikosti 5")

plt.plot(list(range(2009, 2022)), LJ_ss_pet, color='blue', label="Ljubljana srednješolska izobrazba")
plt.plot(list(range(2009, 2022)), MB_ss_pet, color='green', label="Maribor srednješolska izobrazba")
plt.plot(list(range(2009, 2022)), KP_ss_pet, color='red', label="Koper srednješolska izobrazba")
plt.plot(list(range(2009, 2022)), CE_ss_pet, color='orange', label="Celje srednješolska izobrazba")
plt.plot(list(range(2009, 2022)), BL_ss_pet, color='violet', label="Bled srednješolska izobrazba")
plt.legend()
plt.show()



# PRIMERJAVA OSNOVNOŠOLSKE

plt.figure(figsize=(15, 10))
plt.xlabel("Leto")
plt.ylabel("Delež prebivalstva občine")
plt.title("Graf deleža delovno aktivnega prebivalstva občine z osnovnošolsko stopnjo izobrazbe z oknom velikosti 5")

plt.plot(list(range(2009, 2022)), LJ_os_pet, color='blue', label="Ljubljana osnovnošolska izobrazba")
plt.plot(list(range(2009, 2022)), MB_os_pet, color='green', label="Maribor osnovnošolska izobrazba")
plt.plot(list(range(2009, 2022)), KP_os_pet, color='red', label="Koper osnovnošolska izobrazba")
plt.plot(list(range(2009, 2022)), CE_os_pet, color='orange', label="Celje osnovnošolska izobrazba")
plt.plot(list(range(2009, 2022)), BL_os_pet, color='violet', label="Bled osnovnošolska izobrazba")
plt.legend()
plt.show()
